In [14]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import chardet
import pandas as pd
import re
from bs4 import BeautifulSoup



In [16]:
#Methods

import re

def parse_html(text):
    "removes hyperlink from a piece of text"
    
    soup = BeautifulSoup(text, "html.parser")
    parsed_text = soup.get_text()
    return parsed_text

def preproc(text):

    refined_text = re.sub("(\n)", " ", text) #removes python newline 
    refined_text = re.sub("\w+(\')", "" , refined_text) #removes backslash
    refined_text = re.sub("(\s+)", " ", refined_text) #removes whitespaces
    refined_text = re.sub("(&amp|>+|-+)", " ", refined_text)
    refined_text = re.sub("(\d+)", "", refined_text) #removes digits
    refined_text = re.sub("\W*(@)", "", refined_text) #removes emails 

    #refined_text = re.sub("[>|<-\\", " ", )

    return refined_text


def prepare_corpus(path):
    """BY date"""
    corpus = []; temp = []
    for topic in os.listdir(path):
        subfolder = path + '/' + topic
        for doc in os.listdir(subfolder):
            file = subfolder + '/' + doc
            with open(file, 'r', encoding='utf-8', errors= 'ignore') as t:
                corpus.append((preproc(" ".join(t.readlines())),topic),)
    return corpus


def preproc_newsgroup(text):

    refined_text = re.sub("(\n)", " ", text)
    refined_text = re.sub("(\s+)", " ", refined_text)
    refined_text = re.sub("(&amp)", " ", refined_text)

    return

def check_length(text):
    return

def extra_long_documents(text):
    return 

In [17]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "byarticle") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

#change to bypublisher because byarticle has no test set.
article_df = pd.DataFrame(hyperpartisan_dataset['train'])
#article_test_df = pd.DataFrame(hyperpartisan_dataset['test'])

article_text = []

#
train_text_list = list(article_df['text'])
train_tags_list = list(article_df['hyperpartisan'])
train_article_title = list(article_df['title'])

#
#test_text_list = list(article_test_df['text'])
#test_tags_list = list(article_test_df['hyperpartisan'])
#test_article_title = list(article_test_df['title'])

for item in train_text_list:
    article_text.append(preproc(parse_html(item)))
    
len(article_text)

Reusing dataset hyperpartisan_news_detection (/Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/byarticle/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)
100%|██████████| 1/1 [00:00<00:00, 411.00it/s]


645

In [18]:
newsgroup_path = "20news-bydate"
element = os.listdir(newsgroup_path)

newsgroup_train_path = newsgroup_path + '/' + element[1]
newsgroup_test_path = newsgroup_path + '/' + element[0]


In [19]:
newsgroup_train = prepare_corpus(newsgroup_train_path)
newsgroup_test = prepare_corpus(newsgroup_test_path)

len(newsgroup_train), newsgroup_train[0], newsgroup_train[0][0]

X_train_newsgroup_text, y_label_train = zip(*newsgroup_train)
X_test_newsgroup_text, y_label_test = zip(*newsgroup_test)

len(X_train_newsgroup_text),len(X_test_newsgroup_text)

(11314, 7532)

NAIVE BAYES EXPLORATION

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Modelled the conversation with a bag of words approach

In [20]:
X_train_newsgroup_text[2000]

'From: perrydsinc.com (Jim Perry) Subject: Re: [soc.motss, et al.] "Princeton axes matching funds for Boy Scouts" Article I.D.: dsi.pqskINNhi Distribution: usa Organization: Decision Support Inc. Lines:  NNTP Posting Host: dsi.dsinc.com In article <Aprmidway.uchicago.edu  shoumidway.uchicago.edu writes:  In article <pidhINNubdsi.dsinc.com  perrydsinc.com (Jim Perry) writes:  Bigots never concede that their bigotry is irrational; it  is other people who determine that by examining their arguments.  [...]  No! I expected it! ve set yourself up a wonderful little  world where a bigot is whomever you say it is. This is very  comfortable for you imagine, never having to entertain an  argument against your belief system. Simply accuse the person  making of being a bigot. Well, this particular thread of vituperation slopped its venom over into alt.atheism, where we spend most of our time entertaining arguments against our belief system, without resorting to accusing others of bigotry. s somew

In [28]:
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer

#lemmatize = WordNetLemmatizer()
#stemmer = PorterStemmer()


def extract_features(text, method = "Tfidf"):
    """Count represents the number of features to be chosen from tfidf while text represents text data
    vectorizer is either count_vectorizer or tfidfvectorizer"""
    
    if method.lower() == "tfidf":
        vectorizer = TfidfVectorizer()
    elif method.lower() == "count":
        vectorizer = CountVectorizer()
    else:
        print("Wrong method. Choose between tfidf and count")

    vec = vectorizer.fit_transform(text)
    X_features = vectorizer.get_feature_names()
    print(vec.shape)
    return vec, X_features

def accuracy(prediction, goldstandard):
    acc = 0
    for a,b in zip(prediction,goldstandard):
        print(f"{a}----{b}")
        if a == b:
            acc+=1
        else:pass
    return (acc/len(prediction))*100

In [25]:
vec, feat = extract_features(X_train_newsgroup_text, "tfidf")
vec.shape

(11314, 105178)


(11314, 105178)

In [31]:
print(feat)

['__' '___' '____' ... 'zzzz' 'zzzzzz' 'zzzzzzt']


In [ ]:
#Initialising multinomial naive bayes
MNB = MultinomialNB()

#explore other forms of naive bayes

MNB.fit(vec[:, :7000],y_label_train)

#extract features from test data
test_vec,test_features = extract_features(X_test_newsgroup_text)

#Used the first 7000 features in making predictions
predicted_topic = MNB.predict(test_vec[:,:7000])
predicted_topic

accuracy(predicted_topic, y_label_test)

(7532, 78590)
rec.motorcycles----talk.politics.mideast
comp.sys.ibm.pc.hardware----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
rec.sport.hockey----talk.politics.mideast
misc.forsale----talk.politics.mideast
comp.os.ms-windows.misc----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
sci.electronics----talk.politics.mideast
comp.sys.mac.hardware----talk.politics.mideast
comp.graphics----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
sci.space----talk.politics.mideast
comp.windows.x----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
rec.sport.hockey----talk.politics.mideast
misc.forsale----talk.politic

/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


4.500796601168348

SVM

In [ ]:
from sklearn import svm

svc_classifier = svm.SVC(decision_function_shape='ovo')
svc_classifier.fit(vec[:,:7000],y_label_train)
svc_prediction = svc_classifier.predict(test_vec[:,:7000])

accuracy(svc_prediction, y_label_test)

misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale----talk.politics.mideast
misc.forsale

5.177907594264472

BERT fine-tune 

In [16]:
from transformers import BertTokenizer, BertForMaskedLM, BertPreTrainedModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 20)
#training_args = TrainingArguments(output_dir="test_trainer")
#default training arguments 

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(output_dir="./results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)

trainer = Trainer(model = model, 
            args = training_args,
            train_dataset= X_train_newsgroup_text,
            tokenizer= tokenizer,
            data_collator=data_collator)

trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/max/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "labe

AttributeError: 'list' object has no attribute 'keys'